In [34]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import time
import html5lib
import pandas as pd
import numpy as np
import pickle
import BeautifulSoup as soup
from ipy_progressbar import ProgressBar
from IPython.display import HTML

In [20]:
url = 'http://www.bfro.net/GDB/'
pages = {}
r = requests.get(url)
b = soup.BeautifulSoup(r.text)
for link in b.findAll('a'):
    href = link.get('href')
    if href:
        if href.find('/GDB/state_listing.asp')==0 or href.find('/GDB/show_county_reports.asp')==0:
            url2 = 'http://www.bfro.net' + href
            print 'Crawling...' + url2
            st = href.split('=')[1]
            r = requests.get(url2)
            pages[st] = r
            time.sleep(1)

Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ak
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=al
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ar
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=az
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ca
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=co
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ct
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=de
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=fl
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ga
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ia
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=id
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=il
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=in
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ks
Crawling...http://www.bfro.net/GDB/state_listing.asp?state=ky
Crawling

In [21]:
pickle.dump(pages, open('pages_raw.pkl', 'wb'))

In [25]:
pages = pickle.load(open('pages_raw.pkl', 'rb'))

In [193]:
dfs = []
c = 100
for key in pages.keys():
    page = pages[key]
    b = soup.BeautifulSoup(page.text)
    if key.find('ca-')==0:
        reports = b.findAll('li', {'class':'spaced'})
        df = pd.DataFrame({'state': key, 'reports': len(reports), 'county': 'all'}, index=[c])
        c=c+1
        dfs.append(df)
    else:
        tbls = b.findAll('table', {'class': 'countytbl'})
        for tbl in tbls:
            df = pd.read_html(str(tbl))[0]
            df = df[1:]
            df.columns = ['county', 'reports', 'recent', 'last']
            df['state']=key
            df['reports'] = df['reports'].fillna(0)
            df.drop(['recent', 'last'], inplace=True, axis=1)
            dfs.append(df)

df = dfs[0]
for i in range(1, len(dfs)):
    df = df.append(dfs[i])

df.shape

(3105, 3)

In [195]:
df.to_csv('county_level_clean.csv', sep='\t')

In [26]:
reports = []
for key in pages.keys():
    page = pages[key]
    b = soup.BeautifulSoup(page.text)
    for a in b.findAll('a'):
        href = a.get('href')
        if href:
            if href.find('show_county_reports.asp')==0:
                reports.append(href)
            if href.find('show_report.asp')==0:
                reports.append(href)

In [41]:
details = {}

In [42]:
for report in reports:
    url =  'http://www.bfro.net/GDB/' + report
    try:
        x = details[report]
    except:
        print url
        r = requests.get(url)
        time.sleep(1)
        details[report] = r.text

http://www.bfro.net/GDB/show_report.asp?id=46502
http://www.bfro.net/GDB/show_report.asp?id=46206
http://www.bfro.net/GDB/show_report.asp?id=45604
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Accomack
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Amelia
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Amherst
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Augusta
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Bath
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Bedford
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Botetourt
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Brunswick
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Caroline
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Carroll
http://www.bfro.net/GDB/show_county_reports.asp?state=va&county=Chesapeake+City
http://www.bfro.net/GDB/show_county_reports.asp?sta

In [43]:
pickle.dump(details, open('details.pkl', 'wb'))

In [27]:
details = pickle.load(open('details.pkl', 'rb'))

In [55]:
urls = []
for key in details.keys():
    detail = details[key]
    b = soup.BeautifulSoup(detail)
    links = b.findAll('a')
    for link in links:
        href = link.get('href')
        if href is not None and href.find('show_report.asp')==0:
                urls.append(href)

In [61]:
len(detail_pages)

3424

In [63]:
detail_pages = {}
for url in set(urls):
    try:
        check = detail_pages[url]
    except KeyError:
        print url
        r = requests.get('http://www.bfro.net/GDB/' + url)
        detail_pages[url] = r.text
        time.sleep(1)

show_report.asp?id=11771
show_report.asp?id=26196
show_report.asp?id=32696
show_report.asp?id=8376
show_report.asp?id=3822
show_report.asp?id=3820
show_report.asp?id=3826
show_report.asp?id=13729
show_report.asp?id=12802
show_report.asp?id=4676
show_report.asp?id=7718
show_report.asp?id=7954
show_report.asp?id=4672
show_report.asp?id=4859
show_report.asp?id=23018
show_report.asp?id=32634
show_report.asp?id=43823
show_report.asp?id=36618
show_report.asp?id=580
show_report.asp?id=29980
show_report.asp?id=1995
show_report.asp?id=26353
show_report.asp?id=26352
show_report.asp?id=5674
show_report.asp?id=11576
show_report.asp?id=5678
show_report.asp?id=21402
show_report.asp?id=1228
show_report.asp?id=8680
show_report.asp?id=47841
show_report.asp?id=1996
show_report.asp?id=10556
show_report.asp?id=9407
show_report.asp?id=13828
show_report.asp?id=5418
show_report.asp?id=652
show_report.asp?id=1197
show_report.asp?id=1190
show_report.asp?id=1192
show_report.asp?id=9797
show_report.asp?id=9796
s

In [64]:
pickle.dump(detail_pages, open('detail_pages.pkl', 'wb'))

In [66]:
detail_pages = pickle.load(open('detail_pages.pkl', 'rb'))

In [133]:
rows = []
for st in details.keys():
    page = details[st]
    b = soup.BeautifulSoup(page)
    if st.find('show_county_reports.asp')==0:
        continue
    elif st.find('show_report.asp')==0:
        report_num = b.find('span', {'class': 'reportheader'}).text.split('#')[1].strip()
        classification = b.find('span', {'class': 'reportclassification'}).text.replace('(Class ', '').replace(')', '').strip()
        submit_by = b.findAll('span', {'class': 'field'})[0].text.replace('&nbsp;', ' ')
        title = b.findAll('span', {'class': 'field'})[1].text.replace('&nbsp;', ' ')

        year = ''
        season = ''
        month = ''
        date = ''
        state = ''
        county = ''
        location = ''
        town = ''
        road = ''
        also_noticed = ''
        other_witnesses = ''
        other_stories = ''
        time_condition = ''
        environment = ''
        country = 'us'
        ps = b.findAll('p')
        investigator = ''
        for p in ps:
            field = p.find('span', {'class': 'field'})
            if field:
                arr = p.text.split(':')
                if arr[0]=="YEAR":
                    year = arr[1]
                if arr[0]=="SEASON":
                    season = arr[1]
                if arr[0]=="MONTH":
                    month = arr[1]
                if arr[0]=="DATE":
                    date = arr[1]
                if arr[0]=="STATE":
                    state = arr[1]
                if arr[0]=="PROVINCE":
                    state = arr[1]
                    country = 'canada'
                if arr[0]=="COUNTY":
                    county = arr[1]
                if arr[0]=="LOCATION DETAILS":
                    location = arr[1]
                if arr[0]=="NEAREST TOWN":
                    town = arr[1]
                if arr[0]=="NEAREST ROAD":
                    road = arr[1]
                if arr[0]=="ALSO NOTICED":
                    also_noticed = arr[1]
                if arr[0]=="OTHER WITNESSES":
                    other_witnesses = arr[1]
                if arr[0]=="OTHER STORIES":
                    other_stories = arr[1]
                if arr[0]=="TIME AND CONDITIONS":
                    time_condition = arr[1]
                if arr[0]=="ENVIRONMENT":        
                    environment = arr[1]
            slug = 'Follow-up investigation report by '
            if p.text.find(slug) == 0:
                investigator =p.text.replace(slug, '')

        row = [report_num, classification, submit_by, title, year, season, month, date, state, county, country, location, town, road, also_noticed, other_witnesses, other_stories, time_condition, environment, investigator]
        rows.append(row)


In [134]:
print len(rows)

405


In [140]:
df.to_csv('details_cleaned.csv', sep='\t', encoding='utf-8')

In [142]:
df2 = pd.read_csv('details_cleaned.csv', sep='\t')

In [268]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_population')
b = soup.BeautifulSoup(r.text)

In [269]:
df3 = pd.read_html(str(b.findAll('table')[2]))[0]
df3 = df3[[1,6,10]]
df3 = df3[1:]
df3.columns = ['state', 'pop_per_km_2', 'perc_pop']
df3 = df3[df3.state!='Canada']
df3['perc_pop'] = df3['perc_pop'].apply(lambda x: float(x.replace('%', ''))/100)
df3['abbrev'] = ''
df3.ix[df3.state=='Yukon', 'abbrev'] = 'ca-yu'
df3.ix[df3.state=='British Columbia', 'abbrev'] = 'ca-bc'
df3.ix[df3.state=='New Brunswick', 'abbrev'] = 'ca-nb'
df3.ix[df3.state=='Alberta', 'abbrev'] = 'ca-al'
df3.ix[df3.state=='Saskatchewan', 'abbrev'] = 'ca-sk'
df3.ix[df3.state=='Manitoba', 'abbrev'] = 'ca-mn'
df3.ix[df3.state=='Quebec', 'abbrev'] = 'ca-qu'
df3.ix[df3.state=='Ontario', 'abbrev'] = 'ca-on'
canada = df3.copy()

In [270]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_U.S._states_by_population_density')
b = soup.BeautifulSoup(r.text)

In [273]:
df3 = pd.read_html(str(b.findAll('table')[3]))[0]
df3 = df3[[0,2,4]]
df3 = df3[1:]
df3.columns = ['state', 'pop_per_km_2', 'perc_pop']
df3['perc_pop'] = df3['perc_pop'].apply(lambda x: float(x.split('!')[1].replace(',', '')))
df3['pop_per_km_2'] = df3['pop_per_km_2'].apply(lambda x: float(x.split('!')[1].replace(',', '')))
df3['state'] = df3['state'].apply(lambda x: x.split('!')[1].strip())
# convert mi to km
# convert pop to perc
df3 = df3[df3.perc_pop <= 38332521]
df3 = df3[df3.state != 'West South Central']
df3 = df3[df3.state != 'West North Central']
df3 = df3[df3.state != 'East South Central']
df3 = df3[df3.state != 'Mountain']
df3 = df3[df3.state != 'New England']
df3 = df3[df3.state != 'Total Territories']
df3['abbrev'] = ''
df3['perc_pop'] = df3['perc_pop'] / sum(df3['perc_pop'])
df3['pop_per_km_2'] = df3['pop_per_km_2'] * 1.60934

In [275]:
df3['cpop'] = 318900000
canada['cpop'] = 35160000

In [276]:
df4 = df3.append(canada)
df4.to_csv('pop_data.csv', sep='\t', index=False)